# Participants.csv maker

This notebook lets you create a participants.csv file of any [BIDS](https://bids.neuroimaging.io/) folder.

In [4]:
import pandas as pd
import os
from os.path import join 
import glob

DATA_DIR = "/ritter/share/data/IMAGEN"

In [2]:
class PPTS:
    def __init__(self, path):
        self.path = path
        
    def make_df(self):
        files = glob.glob(join(self.path, "**/*_warped.nii.gz"), recursive=True)
        print("There are {} files in the directory.".format(len(files)))
        
        df = pd.DataFrame(columns = ["ID", "time_point", "anat_func", "file_name", "path"])
        
        for i in range(len(files)): 
            file = files[i]
            x = file.split("/")

            df = df.append({
                "path" : files[i],
                "ID" : x[6].replace("sub-", ""), 
                "file_name" : x[-1], 
                "time_point" : x[7], 
                "anat_func" : x[8]
            }
                , ignore_index=True)
            
        self.df = df
    
    def add_center(self):
        tc = pd.read_csv(join(DATA_DIR, "qs/FU3/participants/IMAGEN_demographics.csv"), dtype={"PSC2":str})
        tc = tc.rename(columns={"PSC2" : "ID", "recruitment centre":"center"})

        # Introduce a test centre code 
        d = {
            "BERLIN" : 1, 
            "DRESDEN" : 2,
            "DUBLIN" : 3, 
            "HAMBURG" : 4, 
            "LONDON" : 5, 
            "MANNHEIM" : 6, 
            "NOTTINGHAM" : 7, 
            "PARIS" : 8
        }

        tc["center_code"] = tc["center"].map(d)
        self.df = pd.merge(self.df, tc, how="left")
        
    def add_gender(self):
        gh = pd.read_csv(join(DATA_DIR, "qs/subjects.csv"), sep=";", dtype={"code_in_study":str})
        gh = gh.rename(columns={"code_in_study":"ID"})
        gh = gh[["ID", "gender", "handedness"]]
        #gh = gh.query("handedness == 'right'")
        self.df = pd.merge(self.df, gh, how="left")
        
    def add_y(self, file_names, q_number):
        
        # Create new unit
        df_y = pd.DataFrame(columns=["ID"])
        
        # Open the three ESPAD files 
        for tp, file in zip(["BL", "FU1", "FU2", "FU3"], file_names):
            df = pd.read_csv(join(DATA_DIR, file), dtype={"User code":str})
            df = df.rename(columns={"User code" : "ID"})
            #if "Valid" in list(df):
            #    df = df.query("Valid == 't'")
            df.ID = df.ID.str.replace("-C", "")
            df = df[["ID", q_number]]
            df = df.rename(columns={q_number : "y"+tp})
            df_y = pd.merge(df_y, df, how="outer")
        
        self.df = pd.merge(self.df, df_y, "left")
        
    
    def save_csv(self):
        self.df.to_csv(join(self.path, "participants.csv"), sep=",", index=False)

In [3]:
# The path should be the BIDS folder you want to make a ppts csv for
path = join(DATA_DIR, "IMAGEN_prep-brainmask_BIDS")

ppts = PPTS(path)
ppts.make_df()
ppts.add_gender()
ppts.add_center()
ppts.save_csv()

There are 4680 files in the directory.
